In [1]:
import spacy
from spacy.util import minibatch, compounding
from spacy.training import Example
import random
import json
import os

# Definir os caminhos
data_path = "../../model/training_data/training_data.json"
model_output_dir = "../../model/spaCy_model/"

# Carregar os dados de treinamento
with open(data_path, "r") as file:
    TRAINING_DATA = json.load(file)

# Função para verificar e remover conflitos de sobreposição nos dados de treinamento
def remove_overlapping_entities(data):
    cleaned_data = []
    for text, annotations in data:
        entities = annotations["entities"]
        entities = sorted(entities, key=lambda x: x[0])  # Ordenar as entidades pelo índice inicial
        cleaned_entities = []
        last_end = -1
        for start, end, label in entities:
            if start >= last_end:  # Verificar se não há sobreposição
                cleaned_entities.append((start, end, label))
                last_end = end
        cleaned_data.append((text, {"entities": cleaned_entities}))
    return cleaned_data

# Remover sobreposição dos dados de treinamento
TRAINING_DATA = remove_overlapping_entities(TRAINING_DATA)

# Carregar ou criar um modelo spaCy em branco para NER
nlp = spacy.blank("pt")  # Modelo em português
ner = nlp.add_pipe("ner", last=True)

# Adicionar os rótulos das entidades ao NER
for _, annotations in TRAINING_DATA:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

# Treinamento do modelo
optimizer = nlp.begin_training()
n_iter = 50  # Número de iterações de treinamento

print("Iniciando o treinamento do modelo...")

for itn in range(n_iter):
    random.shuffle(TRAINING_DATA)  # Embaralhar dados a cada iteração
    losses = {}

    # Criar lotes de treinamento com tamanhos variáveis
    batches = minibatch(TRAINING_DATA, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
        examples = []
        for text, annotations in batch:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, {"entities": annotations["entities"]})
            examples.append(example)
        nlp.update(examples, sgd=optimizer, drop=0.5, losses=losses)

    print(f"Iteração {itn + 1}/{n_iter} - Losses: {losses}")

# Salvar o modelo treinado
if not os.path.exists(model_output_dir):
    os.makedirs(model_output_dir)

nlp.to_disk(model_output_dir)
print(f"Modelo de NER treinado e salvo em '{model_output_dir}' com sucesso!")


Iniciando o treinamento do modelo...


/Users/jorgegobbi/programacao/Notas/.venv/lib/python3.10/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Sabão Em Pó Omo Puro Cuidado 2,2kg da marca OMO, n..." with entities "[(0, 34, 'PRODUCT_NAME'), (44, 47, 'BRAND'), (64, ...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


Iteração 1/50 - Losses: {'ner': np.float32(12449.49)}
Iteração 2/50 - Losses: {'ner': np.float32(1637.3721)}
Iteração 3/50 - Losses: {'ner': np.float32(998.2915)}
Iteração 4/50 - Losses: {'ner': np.float32(876.9952)}
Iteração 5/50 - Losses: {'ner': np.float32(735.4539)}
Iteração 6/50 - Losses: {'ner': np.float32(714.20807)}
Iteração 7/50 - Losses: {'ner': np.float32(801.58)}
Iteração 8/50 - Losses: {'ner': np.float32(643.26434)}
Iteração 9/50 - Losses: {'ner': np.float32(647.2936)}
Iteração 10/50 - Losses: {'ner': np.float32(632.73206)}
Iteração 11/50 - Losses: {'ner': np.float32(596.15826)}
Iteração 12/50 - Losses: {'ner': np.float32(560.543)}
Iteração 13/50 - Losses: {'ner': np.float32(635.96423)}
Iteração 14/50 - Losses: {'ner': np.float32(555.2048)}
Iteração 15/50 - Losses: {'ner': np.float32(570.46606)}
Iteração 16/50 - Losses: {'ner': np.float32(573.41736)}
Iteração 17/50 - Losses: {'ner': np.float32(632.7118)}
Iteração 18/50 - Losses: {'ner': np.float32(538.7089)}
Iteração 19/50